In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tscv_sliding
from window_generator import WindowGenerator

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


: 

: 

In [ ]:
from fredapi import Fred
fred_api_key = "29b81578246f3b1d8661dfdb956124ba"
fred = Fred(api_key=fred_api_key)
start_date = "2003-01-02" 
end_date = "2022-12-30"

In [ ]:
vix = fred.get_series("VIXCLS", observation_start=start_date, observation_end=end_date).to_frame()
vix.head()

In [ ]:
vix.rename(columns={0:"VIX"}, inplace=True)
vix

In [ ]:
vix.fillna(method="ffill", inplace=True)

In [ ]:
n = len(vix)
n_train = int(n*0.7)
n_val = int(n*0.9)
train_df = vix[0:n_train]
val_df = vix[n_train:n_val]
test_df = vix[n_val:]

In [ ]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

cols = train_df.columns
train_indices = train_df.index
val_indices = val_df.index
test_indices = test_df.index

scale = MinMaxScaler()
train_df = scale.fit_transform(train_df)
val_df = scale.transform(val_df)
test_df = scale.transform(test_df)

train_df = pd.DataFrame(train_df, columns=cols, index=train_indices)
val_df = pd.DataFrame(val_df, columns=cols, index=val_indices)
test_df = pd.DataFrame(test_df, columns=cols, index=test_indices)
train_df.head()

In [ ]:
input_width = 30

In [ ]:
from sklearn.preprocessing import RobustScaler
w = WindowGenerator(input_width=input_width, 
                    label_width=1,
                    shift=1,
                    train_df=train_df,
                    val_df=val_df,
                    test_df=test_df,
                    n_splits=5,
                    train_splits=3,
                    test_splits=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
def qlike(y_true, y_pred):
  return tf.math.log(y_pred) + (y_true / y_pred)

def build_model():
    model = Sequential()
    model.add(LSTM(32, return_sequences=True, input_shape=(input_width, 1))) #(timesteps, features)
    model.add(LSTM(96, return_sequences=True)) #96 #192 #160
    #model.add(LSTM(256, return_sequences=True)) # #256 #(192)
    model.add(LSTM(64)) #64 
    model.add(Dropout(0.2)) #0.2 #0.1 #0.2
    model.add(Dense(1, activation="relu"))
    
    learning_rate = 0.001 #0.001 #0.0001 #0.001
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(), qlike]
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate), metrics = metrics)
    return model
 

In [ ]:
MAX_EPOCHS = 50
model = build_model()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=5)

In [ ]:
history = model.fit(w.train, epochs=MAX_EPOCHS,
                      validation_data=w.val,
                      callbacks=[early_stopping])

In [ ]:
ar_width = 200
shift = ar_width - input_width

test = np.concatenate([x for x, y in w.test], axis=0)
predictions = model.predict(test)
y = np.concatenate([y for x, y in w.test], axis=0).reshape(-1, 1)

predicted = scale.inverse_transform(predictions).flatten()[shift:]
actual = scale.inverse_transform(y).flatten()[shift:]
predicted.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
timesteps = np.array([i for  i in range(len(predicted))])
n = 322
sns.lineplot(ax = ax, x = timesteps[0:n], y = predicted[0:n], label = 'LSTM')
sns.lineplot(ax = ax, x = timesteps[0:n], y = actual[0:n], label = 'VIX')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse =  mean_squared_error(actual, predicted)
mae =  mean_absolute_error(actual, predicted)

print("MSE: ", mse)
print("MAE: ", mae)



Econometric benchmark

In [ ]:
input_width_ar = ar_width

ar_w = WindowGenerator(input_width=input_width_ar, 
                    label_width=1,
                    shift=1,
                    train_df=train_df,
                    val_df=val_df,
                    test_df=test_df,
                    n_splits=3,
                    train_splits=3,
                    test_splits=1,
                    scale=False)

In [ ]:
y_test = np.concatenate([y for x, y in ar_w.test], axis=0)
x_test = np.concatenate([x for x, y in ar_w.test], axis=0)

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

predictions = []
resid = []

for x, y in zip(x_test, y_test):
    arima_model = AutoReg(endog=x, lags=3).fit()
    day_ahead = x_test.shape[1] + 1
    predictions.append(arima_model.predict(day_ahead, day_ahead))
    resid.append(y[0] - arima_model.predict(day_ahead, day_ahead))

In [ ]:
dates = np.arange(1, len(predictions) + 1)

y = scale.inverse_transform(y_test.reshape((len(predictions), -1)))
predictions = scale.inverse_transform(np.array(predictions))
print(y.shape)


dates = test_indices[input_width:]
print(dates.shape)

fig, ax = plt.subplots(figsize = (25,10))

n = 322 # How many days to see in plot

plt.plot(dates[:n], predictions[:n], label = "AR(3)")
plt.plot(dates[:n], y[:n], label = "VIX")
plt.title("Out of sample forecasts: one day ahead")
plt.legend()

In [ ]:
mse = mean_squared_error(y, predictions)
mae = mean_absolute_error(y, predictions)

print("MSE: ", mse)
print("MAE: ", mae)

Bayesian model

In [ ]:
import tensorflow_probability as tfp
tfd = tfp.distributions

negloglik = lambda y, rv_y: -rv_y.log_prob(y)

n_samples = np.concatenate([x for x, y in w.train], axis=0).shape[0]

# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  c = np.log(np.expm1(1.))
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
          reinterpreted_batch_ndims=1)),
  ])

# Specify the prior over `keras.layers.Dense` `kernel` and `bias`.
def prior_trainable(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
  ])

In [ ]:
def build_bnn():
    model = Sequential()
    model.add(LSTM(32, return_sequences=True, input_shape=(input_width, 1))) #(timesteps, features)
    model.add(LSTM(96, return_sequences=True)) #96 #192 #160
    #model.add(LSTM(256, return_sequences=True)) # #256 #(192)
    model.add(LSTM(64)) #64 
    model.add(Dropout(0.2)) #0.2 #0.1 #0.2
    model.add(tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight=1/n_samples))
    #model.add(Dense(1, activation="relu"))
    
    learning_rate = 0.001 #0.001 #0.0001 #0.001
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(), qlike]
    model.compile(loss=negloglik, optimizer=Adam(learning_rate=learning_rate), metrics = metrics)
    return model

In [ ]:
bnn = build_bnn()
bnn.summary()

In [ ]:
MAX_EPOCHS = 300
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=5)

In [ ]:
history = bnn.fit(w.train, epochs=MAX_EPOCHS,
                      validation_data=w.val)
                      #callbacks=[early_stopping])

In [ ]:
b_x = np.concatenate([x for x, y in w.test], axis = 0)
b_y = np.concatenate([y for x, y in w.test], axis = 0)

ensemble = []

for i in range(100):
    ensemble.append(bnn(b_x))

ensemble = scale.inverse_transform(np.array(ensemble).reshape((100, -1)))

mean = np.mean(ensemble, axis = 0)
std = np.std(ensemble, axis = 0)

b_y = scale.inverse_transform(b_y.reshape((len(mean), -1)))


In [ ]:
b_y.shape

In [ ]:
mse = mean_squared_error(b_y, mean)
mse

In [ ]:
test_df = vix[n_val:]

mean = mean.flatten()
b_y = b_y.flatten()


preds_interval = pd.DataFrame(ensemble.T, index = test_indices[30:])
preds_interval['Date'] = test_indices[30:]
preds_interval = preds_interval.melt(id_vars = 'Date', var_name = 'Labels', value_name = 'Vals')

timesteps = np.array([i for  i in range(len(mean))])

actual_interval = pd.DataFrame([test_indices[30:], b_y]).T
actual_interval.columns = ["Date", "Vals"]


n = 300

fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(ax = ax, data = preds_interval, x = 'Date', y = 'Vals', color = "r", label="BNN", errorbar=("sd"))
sns.lineplot(ax = ax, data = test_df)
plt.legend()
plt.show()

In [ ]:
resid = np.abs(b_y - mean)

sns.regplot(x = resid, y = std)
plt.xlabel("Prediction error")
plt.ylabel("Prediction standard deviation")